In [1]:
# Training script for LazyConfig models
try:
    # ignore ShapelyDeprecationWarning from fvcore
    import warnings

    from shapely.errors import ShapelyDeprecationWarning

    warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

except:
    pass
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger

setup_logger()

import gc
import os
import time

import detectron2.utils.comm as comm

# import some common libraries
import numpy as np
import torch

# import some common detectron2 utilities
from detectron2.config import LazyConfig, get_cfg
import detectron2.data as data
from detectron2.engine import (
    launch,
)

from deepdisc.data_format.augment_image import train_augs
from deepdisc.data_format.image_readers import DC2ImageReader
from deepdisc.data_format.register_data import (
    register_data_set,
)  # , register_loaded_data_set
from deepdisc.model.loaders import (
    RedshiftFlatDictMapper,
    return_test_loader,
    return_train_loader,
)
from deepdisc.model.models import (
    RedshiftPointCasROIHeads,
    RedshiftPointROIHeads,
    RedshiftPDFROIHeads,
    return_lazy_model,
)
from deepdisc.training.trainers import (
    return_evallosshook,
    return_lazy_trainer,
    return_optimizer,
    return_savehook,
    return_schedulerhook,
)
from deepdisc.utils.parse_arguments import make_training_arg_parser

/home/drewoldag/.conda/envs/rail_deepdisc39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import importlib
# importlib.reload(rail.estimation.algos.deepdisc)

In [3]:
# from rail.estimation.algos.deepdisc import DeepDiscInformer
import rail
from rail.estimation.algos.deepdisc import *
from rail.core.data import TableHandle, JsonHandle
from rail.core.stage import RailStage

from rail.deepdisc.configs import *

In [4]:
#/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_train_small.npy
cfgfile = "/home/shared/hsc/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py"
dirpath = "./tests/deepdisc/test_data/dc2/"
output_dir = "./"
output_name = "test"

#trainfile = dirpath + "flattened_data_test.npy"
trainfile = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_train.hdf5"
testfile = dirpath + "flattened_data_test.npy"
metadatafile = "/home/shared/hsc/DC2/test_data/dataset_3/train_metadata.json"
classes = ["object"]
numclasses = 1

In [5]:
cfg = get_lazy_config(cfgfile, 1, 1)
cfg.train.init_checkpoint

'detectron2://ImageNetPretrained/MSRA/R-50.pkl'

In [6]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [7]:
from deepdisc.data_format.file_io import get_data_from_json

#testdata = np.load(trainfile, allow_pickle=True)
testdata = DS.read_file("testdata", TableHandle, trainfile)
metadata = get_data_from_json(metadatafile)

In [8]:
#testdata = np.load("./tests/deepdisc/test_data/dc2/flattened_data_test.npy")
mapper = RedshiftFlatDictMapper().map_data

In [9]:
testdata()

OrderedDict([('images',
              array([[-0.12334768, -0.02265165, -0.08154224, ...,  0.11944132,
                       0.26857075, -0.44378543],
                     [-0.12334768, -0.02265165, -0.08154224, ...,  0.11944132,
                       0.26857075, -0.44378543],
                     [-0.12334768, -0.02265165, -0.08154224, ...,  0.11944132,
                       0.26857075, -0.44378543],
                     ...,
                     [-0.12334768, -0.02265165, -0.08154224, ...,  0.11944132,
                       0.26857075, -0.44378543],
                     [-0.12334768, -0.02265165, -0.08154224, ...,  0.11944132,
                       0.26857075, -0.44378543],
                     [-0.12334768, -0.02265165, -0.08154224, ...,  0.11944132,
                       0.26857075, -0.44378543]], dtype=float32))])

In [10]:
# dataset_dicts = {}
# dds = []
# for row in testdata:
#     dds.append(mapper(row))
# dataset_dicts["test"] = dds

In [11]:
training = DS.add_data("training", testdata(), TableHandle, path=trainfile) #()["images"]
testing = DS.add_data("testing", testdata, TableHandle)

metadatahandle = DS.add_data("metadata", metadata, JsonHandle, path=metadatafile)

In [12]:
import h5py

f = h5py.File(trainfile, 'r')

f.keys()

<KeysViewHDF5 ['images']>

In [13]:
training

In [14]:
print(testdata)

<class 'rail.core.data.TableHandle'> /home/shared/hsc/DC2/test_data/dataset_3/flattened_images_train.hdf5, (wd)


In [15]:
deep_dict = dict(
    chunk_size=100,
    epochs=200,
    numclasses=1,
    batch_size=1,
    output_dir="./",
    cfgfile="/home/shared/hsc/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py",
    output_name="test_informer",
#     hdf5_groupname="images",
    
)

In [16]:
# Inform = DeepDiscInformer.make_stage(name='Inform_DeepDISC', model='detectron2://ImageNetPretrained/MSRA/R-50.pkl', **deep_dict)
Inform = DeepDiscInformer.make_stage(
    name="Inform_DeepDISC", model="test_informer.pkl", **deep_dict
)

In [17]:
Inform.inform(training, metadatahandle)

[12/06 15:13:11 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[12/06 15:13:11 d2.data.common]: Serializing 1000 elements to byte tensors and concatenating them all ...
[12/06 15:13:22 d2.data.common]: Serialized dataset takes 274.77 MiB
[12/06 15:13:22 d2.data.build]: Making batched data loader with batch_size=1
[12/06 15:13:23 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[12/06 15:13:23 d2.data.common]: Serializing 1000 elements to byte tensors and concatenating them all ...
[12/06 15:13:33 d2.data.common]: Serialized dataset takes 274.77 MiB
[12/06 15:13:34 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from detectron2://ImageNetPretrained/MSRA/R-50.pkl ...
[12/06 15:13:34 d2.checkpoint.c2_model_loading]: Renaming Caffe2 weights ......
WARNING [12/06 15:13:34 d2.checkpoint.c2_model_loading]: Shape of stem.conv1.weight in checkpoint is torch.Size([64, 

Some model parameters or buffers are not found in the checkpoint:
backbone.bottom_up.stem.conv1.weight
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.deconv.{bias, weight}
roi_heads.mask_head.mask_fcn1.{bias, weight}
roi_heads.mask_head.mask_fcn2.{bias, weight}
roi_heads.mask_head.mask_fcn3.{bias, weight}
roi_heads.mask_head.mask_fcn4.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight

Model training:
[12/06 15:13:34 d2.engine.train_loop]: Starting training from iteration 0
Iteration:  5  time:  2.260785549879074e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.5425099730491638, 0.21780839562416077, 0.10481711477041245, 0.5889551639556885, 0.23823335766792297] val loss:  0 lr:  [0.001]
Iteration:  10  time:  1.5902332961559296e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.5010473728179932, 0.17743214964866638, 0.09842608124017715, 0.36248016357421875, 0.24596776068210602] val loss:  0 lr:  [0.001]
Iteration:  15  time:  2.759043127298355e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.808623194694519, 0.36287716031074524, 0.09668457508087158, 0.2638605237007141, 0.23049607872962952] val loss:  0 lr:  [0.001]
Iteration:  20  time:  3.110617399215698e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', '

### Inference

In [14]:
# Estimator = DeepDiscEstimator.make_stage(name='DeepDiscEstimator',
#                                       model=Inform.get_handle('model'), **deep_dict)

Estimator = DeepDiscPDFEstimator.make_stage(
    name="DeepDiscEstimator",
    model=Inform.get_handle("model"),
    hdf5_groupname=None,
    **deep_dict,
)

In [15]:
results = Estimator.estimate(testing)

[12/01 16:58:02 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/g4merz/deepdisc/Swin_test.pth ...
Processing Data
Matching objects
Inserting handle into data store.  output_DeepDiscEstimator: inprogress_output_DeepDiscEstimator.hdf5, DeepDiscEstimator


In [16]:
res = results.read()

In [17]:
res